In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import os
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [C:\Users\jglen\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [11]:
# Visit NASA
url = 'https://redplanetscience.com/'
browser.visit(url)
time.sleep(1)

In [12]:
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
headline = soup.find('div', class_='content_title').get_text()
teaser = soup.find('div', class_='article_teaser_body').get_text()
print(headline)
print(teaser)

NASA Perseverance Mars Rover Scientists Train in the Nevada Desert
Team members searched for signs of ancient microscopic life there, just as NASA's latest rover will on the Red Planet next year.


In [16]:
# visit the JPL website and scrape the featured image
url = 'https://spaceimages-mars.com'
browser.visit(url)
time.sleep(1)
soup = bs(browser.html, 'html.parser')

In [17]:
image_src = soup.find_all('img')[1]["src"]
featured_image_url = url + image_src

In [18]:
# visit Mars Facts and scrape the Mars Facts table
mars_facts_url = 'https://galaxyfacts-mars.com'
browser.visit(mars_facts_url)
time.sleep(1)
soup = bs(browser.html, 'html.parser')
soup()

[<html><head>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
 <link href="css/font.css" rel="stylesheet" type="text/css"/>
 <link href="css/app.css" rel="stylesheet" type="text/css"/>
 <link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
 <title>Mars Facts - Interesting facts about Planet Mars</title>
 </head>
 <body>
 <nav class="navbar navbar-expand-lg navbar-light fixed-top">
 <div class="container">
 <a class="navbar-brand" href="#"> <img src="image/logo.png"/>
 </a>
 <button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-bs-target="#navbarNav" data-bs-toggle="collapse" type="button">
 <span class="navbar-toggler-icon"></span>
 </button>
 <div class="collapse navbar-collapse justify-content-end" id="n

In [20]:
mars_df = pd.read_html(browser.html)[1]
mars_facts_html = mars_df.to_html()

In [21]:
# Mars Hemispheres
mars_hemisphere_url = 'https://marshemispheres.com/'
browser.visit(mars_hemisphere_url)
time.sleep(1)
soup = bs(browser.html, 'html.parser')

In [22]:
links = soup.find_all('a', class_='itemLink')

image_data = []
for l in links:
    try:
        browser.visit(mars_hemisphere_url + l['href'])
        time.sleep(1)
        soup = bs(browser.html, 'html.parser')
        hemi_title = soup.find('h2', class_='title')
        hemi_link = soup.find('img', class_='wide-image')['src']
        my_dict = {'title': hemi_title.get_text(), 'img_url': hemi_link}
        image_data.append(my_dict)
    except TypeError:
        print('Error')

Error


In [23]:
hemisphere_image_urls = []
for element in image_data:
    if element not in hemisphere_image_urls:
        hemisphere_image_urls.append(element)
        
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [28]:
# Close browser
browser.quit()